# 1. Read Database
## 1.1. Read derlem.txt

In [34]:
import os, re
import codecs
import collections

'''
Transfer DERLEM to a Dictionary.
'''
with open("derlem.txt", encoding="utf-16") as f:
    opened_file = f.readlines()
    
file_dict = {}
for i, line in enumerate(opened_file):
    if line != "\n":
        '''
        The first number in the line is the Paragraph ID,
        assign the Paragraph ID as the key to the dictionary,
        and the rest of the paragraph to the value.
        '''
        file_dict[int(line.split(' ', 1)[0])] = re.findall(" (.*)", line)[0]

## Read soru_gruplari.txt

In [57]:
# Change the encoding of the input file to latin-1
file_content = ""
with open("soru_gruplari.txt", encoding="utf-16") as f:
    file_content = f.read()
# Read the file and append all of the lines
question_groups = file_content.split("\n\n")

In [61]:
question_groups_dict = {}
for group in question_groups:
    lines = group.split("\n")
    answer_id = ""
    group_dict = collections.defaultdict(dict)
    for line in lines:
        ID = re.findall("(.*):", line)[0]
        sentence = re.findall(": (.*)", line)[0]
        if ID[0] == 'S':  # if the sentence is a question
            group_dict["questions"][ID] = sentence
        elif ID[0] == 'C':  # if the sentence is the answer
            group_dict["answer"] = sentence
            answer_id = ID
        else:  # if the sentence is the id of related paragraph
            group_dict["paragraph"] = sentence
    question_groups_dict[answer_id] = group_dict